In [1]:
import sys
sys.path.append('..')
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

# Train VIME-Semi
from src.VIME.vime_semi import vime_semi
# Train VIME-Self
from src.VIME.vime_self import vime_self

from sklearn.metrics import accuracy_score


2024-06-09 13:02:56.813554: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Experimental parameters
label_no = 1000  
model_sets = ['logit','xgboost','mlp']
  
# Hyper-parameters from original
p_m = 0.3
alpha = 2.0
K = 3
beta = 1.0
label_data_rate = 0.1

# Metric
metric = 'acc'
  
# Define output
results = np.zeros([len(model_sets)+2])  

In [3]:
import numpy as np
from src.data_loader import *
from src.utils import (
    append_acc_early_termination,
    get_train_test_unlabeled,
    get_train_test_unlabeled_for_multilabel,
)
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import os
from scipy.stats import sem


sweeps = [('seer', '1.0'), ('adult', '0.66'), ('cutract', '1.0'), ('covid', '0.66'),   ('maggic', '0.66'), ("compas", "1.0"), ("agaricus-lepiota", '1.0'), ('German-credit', '1.0'), ("higgs", "0.1"),   ('drug', '0.1'), ("blog", "0.2"),("credit", "1.0")]
sweeps = [('seer', '1.0')]

seeds = [0, 1, 2] # List of seeds
nest = 100
prop_lab = 0.1

vime_res = {}
failed = []

for sweep in sweeps:
    dataset_name, prop_data = sweep[0], float(sweep[1])
    acc_scores = []  # List to store accuracy scores for each seed

    for seed in seeds:
        try:
            dataset_name, prop_data = sweep[0], float(sweep[1])


            if dataset_name in ["seer", "cutract", "covid", "support", "adult", "bank", "drug", "metabric", "credit", "maggic", "higgs", "blog", "compas"]:
                            df_feat, df_label, df = get_data(dataset=dataset_name, prop=prop_data)


            else:
                path_to_file = "./data/all_data.pickle"
                (
                    x_train,
                    y_train,
                    x_test,
                    y_test,
                    x_unlabeled,
                    y_unlabeled,
                ) = get_train_test_unlabeled(dataset_name, prop_data-0.0001, path_to_file, random_state=seed)

                #print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_unlabeled.shape, y_unlabeled.shape)
                total_samples = len(x_train) + len(x_test) + len(x_unlabeled)


            x_train, x_test, y_train, y_test = train_test_split(
                                df_feat, df_label, test_size=0.2, random_state=seed
                            )

            x_train, x_unlabeled, y_train, y_unlabeled = train_test_split(
                x_train, y_train, train_size=prop_lab, random_state=seed
            )

            print(dataset_name, len(x_train), len(x_unlabeled), len(x_test))

            x_train = x_train.values
            x_unlab = x_unlabeled.values
            x_test = x_test.values
            y_train = y_train.values
            y_test = y_test.values

    
            vime_self_parameters = dict()
            vime_self_parameters['batch_size'] = 128
            vime_self_parameters['epochs'] = 10
            vime_self_encoder = vime_self(x_unlab, p_m, alpha, vime_self_parameters)
            
            # Save encoder
            if not os.path.exists('save_model'):
                os.makedirs('save_model')

            file_name = './encoder_model.h5'
            
            vime_self_encoder.save(file_name) 


            vime_semi_parameters = dict()
            vime_semi_parameters['hidden_dim'] = 100
            vime_semi_parameters['batch_size'] = 128
            vime_semi_parameters['iterations'] = 1000
            y_test_hat = vime_semi(x_train, y_train, x_unlab, x_test, 
                                vime_semi_parameters, p_m, K, beta, file_name)


            y_pred = y_test_hat.flatten() > 0.5
            acc_scores.append(accuracy_score(y_pred, y_test))

        except Exception as e:
            print(f"Error processing {dataset_name} with seed {seed}: {e}")
            failed.append((dataset_name, seed))
            break  # Breaks the current seed loop, moves to the next dataset

    # After running all seeds, calculate and store the statistics
    if acc_scores:
        vime_res[dataset_name] = {
            "mean": np.mean(acc_scores),
            "std": np.std(acc_scores),
            "sem": sem(acc_scores)  # sem from scipy.stats
        }

# Print the results and failed datasets
print("VIME Results:", vime_res)
print("Failed datasets:", failed)


seer 1600 14400 4000
Epoch 1/10
113/113 [==============================] - 1s 2ms/step - loss: 1184.4727 - mask_loss: 2.2677 - feature_loss: 591.1025
Epoch 2/10
113/113 [==============================] - 0s 1ms/step - loss: 1177.0480 - mask_loss: 0.5979 - feature_loss: 588.2249
Epoch 3/10
113/113 [==============================] - 0s 1ms/step - loss: 1176.6344 - mask_loss: 0.3689 - feature_loss: 588.1328
Epoch 4/10
113/113 [==============================] - 0s 1ms/step - loss: 1176.4617 - mask_loss: 0.3550 - feature_loss: 588.0533
Epoch 5/10
113/113 [==============================] - 0s 1ms/step - loss: 1176.4022 - mask_loss: 0.3535 - feature_loss: 588.0242
Epoch 6/10
113/113 [==============================] - 0s 1ms/step - loss: 1176.3917 - mask_loss: 0.3532 - feature_loss: 588.0193
Epoch 7/10
113/113 [==============================] - 0s 1ms/step - loss: 1176.3834 - mask_loss: 0.3532 - feature_loss: 588.0150
Epoch 8/10
113/113 [==============================] - 0s 1ms/step - loss: 11

In [4]:
vime_res

{'seer': {'mean': 0.8015833333333333,
  'std': 0.007400262758097803,
  'sem': 0.00523277597881322}}

In [5]:
import pickle
filename = f"../results/vime_res.p"

with open(filename, 'wb') as f:
    pickle.dump(vime_res, f)